In [24]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

In [25]:
load_dotenv()
llm = ChatOpenAI()

In [26]:
class jokeState(TypedDict):
    topic: str 
    joke: str 
    explanation: str 

In [27]:
def generate_joke(state: jokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [28]:
def generate_explanation(state: jokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt)

    return {'explanation': response}

In [29]:
state = StateGraph(jokeState)

state.add_node('generate_joke', generate_joke)
state.add_node('generate_explanation', generate_explanation)

state.add_edge(START, 'generate_joke')
state.add_edge('generate_joke', 'generate_explanation')
state.add_edge('generate_explanation', END)

workflow = state.compile()

In [30]:
initial_state = {
    'topic': "Create a joke about monkey"
}

workflow.invoke(initial_state)

{'topic': 'Create a joke about monkey',
 'joke': 'Why did the monkey like to hang out with the banana?\n\nBecause they were both peeling good!',
 'explanation': AIMessage(content='This joke is a play on words, using the pun of "peeling" to mean both feeling good and the act of peeling a banana. In this case, the monkey enjoys hanging out with the banana because they are both "peeling good" in the sense that they are feeling good and also physically peeling, or removing the skin from, the banana. It\'s a light-hearted and silly joke that plays on the similarities between the monkey and the banana.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 34, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.